In [1]:
import pandas as pd, numpy as np, geopandas as gpd
import psycopg2
from shapely.geometry import Point

## Preparing an FVS input database from FIA data

The Pacific Northwest Forest Inventory and Analysis Database (PNW-FIADB) contains repeated field measurements of forest conditions on permament plots collected by the USDA Forest Service across California, Oregon, and Washington. 

The database version used in this study contains data recorded from 1999-2016, and is accessible online [here](https://www.fs.fed.us/pnw/rma/fia-topics/inventory-data/).

To facilitate the utilization of FIA data for forest modeling, FIA data are converted into a PostgreSQL database that can be accessed by the [Forest Vegetation Simulator (FVS)](https://www.fs.fed.us/fvs/) growth-and-yield model. 

FVS can generate a wide variety of metrics useful for forestry analyses. The use of databases (instead of FVS-formatted text files) for input and output data is described in the [User's Guide](https://www.fs.fed.us/fmsc/ftp/fvs/docs/gtr/DBSUserGuide.pdf) to the FVS Database Extension. Additional guidance for the translation of FIA data into FVS can be found in [Shaw (2009)](https://www.fs.fed.us/fmsc/ftp/fvs/docs/gtr/DBSUserGuide.pdf). 

The most recent version of the PNW FIA data were downloaded as a Microsoft Access database, and the following tables were exported to a PostgreSQL database: `plot`, `cond`, `subplot`, `tree`, and `seedling` (all column headers were subsequently converted to lowercase). 

This notebook works through the conversion of FIA data into two input tables that will be used in FVS growth-and-yield modeling: `standinit` and `treeinit`.

## We first need to identify which FVS location codes to use for each FIA plot  
FVS is organized into several regional "variants", each which contains parameterizations for tree growth among several sub-regional location codes. A shapefile containing the FVS variants and locations can be downloaded [here](https://www.fs.fed.us/fmsc/ftp/fvs/docs/overviews/FvsVariantMap20160308.zip).

<img src="https://www.fs.fed.us/fvs/images/FvsVariantsFig1.gif" align="left"><br clear='all'>

#### To identify the FVS variant and location that each FIA plot falls within, we will:
1. Read in the FIA plot locations and FVS variant geospatial data;
2. Convert the LAT and LON coordinates of the plots into points we can use for geospatial analysis;
3. Reproject the points to the same coordinate reference system as the shapefile of the FVS variants;
4. Perform a spatial join; and
5. Format and write our data to a new `fvs_locations` table in the database.

In [3]:
# read in the shapefile for the FVS variants and take a look
fvs_variants = gpd.read_file('FVS_Variants_and_Locations.shp')
fvs_variants.head()

,FVSLocCode,FVSLocName,FVSVarName,FVSVariant,OBJECTID,Shape_Area,Shape_Leng,geometry
0,919,Allegheny National Forest,Northeast,NE,1,1.502990e+11,2.266662e+06,"POLYGON ((1686733.967900001 2292312.427999999,..."
1,501,Angeles National Forest,Central Rockies,CR,2,9.877439e+06,5.476662e+04,"(POLYGON ((-1958769.343699999 1483962.4937, -1..."
2,501,Angeles National Forest,Western Sierra Nevada,WS,3,8.010051e+09,4.658106e+05,"POLYGON ((-2034507.8123 1539322.2918, -2033735..."
3,301,Apache National Forest,Central Rockies,CR,4,1.226670e+10,6.676861e+05,"POLYGON ((-1200179.522 1344226.8469, -1196643...."
4,201,Arapaho National Forest,Central Rockies,CR,5,6.191976e+09,5.051997e+05,POLYGON ((-826763.1062000003 1979989.555500001...


In [2]:
# how to connect to our database and get the data
pg_engine='postgresql://postgres@localhost:5432/PNWFIADB_FVSIn'
SQL = 'SELECT plot.cn AS plt_cn, plot.lat, plot.lon FROM plot'

# read in the stands from the FVSIn database
plots = pd.read_sql(sql=SQL, con=pg_engine)

plots.head()

,plt_cn,lat,lon
0,10775507020004,43.045378,-123.905345
1,12383631010497,37.389432,-118.813089
2,12383786010497,36.802111,-118.301728
3,12384680010497,36.924149,-118.881227
4,12385290010497,37.276415,-118.662680


In [6]:
# use Shapely's Point function to create geospatial points
geometry = [Point(xy) for xy in zip(plots.lon, plots.lat)]

# create a geodataframe using the WGS 84 coordinate reference system
plots = gpd.GeoDataFrame(plots, crs={'init': 'epsg:4326'}, geometry=geometry)

# reproject plot points to same coordinate reference system as FVS variants shapefile
plots = plots.to_crs(fvs_variants.crs)

# perform the spatial join taking the columns we want from the FVS variants shapefile
columns = ['geometry', 'FVSLocCode', 'FVSLocName', 'FVSVarName', 'FVSVariant']
plots_with_locations = gpd.sjoin(plots, fvs_variants[columns], how='inner', op='intersects').drop('index_right', axis=1)

# convert column names to lower case
plots_with_locations.columns = plots_with_locations.columns.str.lower()
plots_with_locations.head()

,plt_cn,lat,lon,geometry,fvsloccode,fvslocname,fvsvarname,fvsvariant
0,10775507020004,43.045378,-123.905345,POINT (-2228996.692427667 2557402.232319243),712,BLM - Coos Bay District,Pacific Northwest Coast,PN
192,12823249010497,43.795479,-123.104075,POINT (-2143205.636505469 2619224.358320495),712,BLM - Coos Bay District,Pacific Northwest Coast,PN
277,12843390010497,43.089221,-123.638670,POINT (-2206898.225915615 2555867.982977629),712,BLM - Coos Bay District,Pacific Northwest Coast,PN
285,12845801010497,43.235800,-123.511794,POINT (-2192379.487177448 2568617.224989019),712,BLM - Coos Bay District,Pacific Northwest Coast,PN
287,12846451010497,43.252741,-123.685232,POINT (-2205265.252012831 2574442.674130918),712,BLM - Coos Bay District,Pacific Northwest Coast,PN


In [7]:
# if you don't have the database setup to handle GIS data types, 
# you'll get an error trying to write the geometry column to the database
plots_with_locations.drop('geometry', axis=1).to_sql('fvs_locations', pg_engine, index=False, if_exists='replace')

## We will next convert FIA data into the format that FVS expects 
As described in the User Guide FVS Database Extension's [User's Guide](https://www.fs.fed.us/fmsc/ftp/fvs/docs/gtr/DBSUserGuide.pdf), there are a variety of stand- and tree-level inputs that FVS requires which can be read from a database using the `STANDSQL` and `TREESQL` keywords. Rather than performing these elaborate SQL queries at FVS run-time, we will pre-format the data to allow for quicker querying during growth-and-yield runs.

In [92]:
standinit_sql = '''
SELECT 
subplot.cn AS stand_id, 
plot.measyear AS inv_year, 
fvs_locations.fvsvariant AS variant, 
plot.lat AS latitude, 
plot.lon AS longitude, 
fvs_locations.fvsloccode AS location, 
plot.ecosubcd as ecoregion,
cond.habtypcd1 AS pv_code, 
CASE WHEN cond.stdage = 9999 THEN NULL ELSE cond.stdage END AS age, 
subplot.aspect AS aspect, 
subplot.slope AS slope, 
plot.elev AS elevft, 
-1 AS basal_area_factor, 
1 AS inv_plot_size, 
999 AS brk_dbh, 
1 AS num_plots, 
0 AS dg_trans, 
10 AS dg_measure, 
0 AS htg_trans, 
5 AS htg_measure, 
10 AS mort_measure, 
cond.fortypcd AS forest_type, 
plot.statecd AS state, 
plot.countycd AS county
FROM subplot, plot, fvs_locations, cond
WHERE subplot.plt_cn = plot.cn AND plot.cn = fvs_locations.plt_cn AND 
subplot.plt_cn = cond.plt_cn AND subplot.macrcond = cond.condid
'''

standinit = pd.read_sql(sql=standinit_sql, con=pg_engine)

# convert stand_id to an integer
standinit['stand_id'] = standinit['stand_id'].astype('int64')

standinit.head()

,stand_id,inv_year,variant,latitude,longitude,location,ecoregion,pv_code,age,aspect,...,brk_dbh,num_plots,dg_trans,dg_measure,htg_trans,htg_measure,mort_measure,forest_type,state,county
0,12384691010497,2004,WS,36.924149,-118.881227,515,M261Eq,None,NaN,20.0,...,999,1,0,10,0,5,10,NaN,6,19
1,12384696010497,2004,WS,36.924149,-118.881227,515,M261Eq,None,NaN,20.0,...,999,1,0,10,0,5,10,NaN,6,19
2,12384701010497,2004,WS,36.924149,-118.881227,515,M261Eq,None,NaN,35.0,...,999,1,0,10,0,5,10,NaN,6,19
3,12384706010497,2004,WS,36.924149,-118.881227,515,M261Eq,None,NaN,40.0,...,999,1,0,10,0,5,10,NaN,6,19
4,12385305010497,2004,WS,37.276415,-118.662680,504,M261Eo,None,NaN,NaN,...,999,1,0,10,0,5,10,NaN,6,27


In [93]:
standinit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 269800 entries, 0 to 269799
Data columns (total 24 columns):
stand_id             269800 non-null int64
inv_year             269800 non-null int64
variant              269800 non-null object
latitude             269800 non-null float64
longitude            269800 non-null float64
location             269800 non-null int64
ecoregion            269800 non-null object
pv_code              97416 non-null object
age                  118072 non-null float64
aspect               132466 non-null float64
slope                132465 non-null float64
elevft               269800 non-null int64
basal_area_factor    269800 non-null int64
inv_plot_size        269800 non-null int64
brk_dbh              269800 non-null int64
num_plots            269800 non-null int64
dg_trans             269800 non-null int64
dg_measure           269800 non-null int64
htg_trans            269800 non-null int64
htg_measure          269800 non-null int64
mort_measure     

In [94]:
standinit.to_sql('fvs_standinit', pg_engine, index=False, if_exists='replace')

In [4]:
treeinit_sql = '''
SELECT 
subplot.cn as stand_id, 
subplot.subp AS plot_id, 
tree AS tree_id, 
CASE WHEN tree.statuscd = 1 THEN tpa_unadj WHEN tree.statuscd = 2 
THEN tpamort_unadj WHEN tree.statuscd = 3 THEN tparemv_unadj END AS tree_count, 
CASE WHEN tree.statuscd = 1 THEN 1 WHEN (tree.statuscd = 2 AND 
((plot.measyear - tree.mortyr) >10)) THEN 8 WHEN (tree.statuscd > 1 AND 
((plot.measyear - tree.mortyr) <=10)) THEN 6 END AS history, 
spcd AS species, 
CASE WHEN tree.statuscd = 1 THEN dia WHEN tree.statuscd > 1 THEN diacalc END AS dbh, 
ROUND(tree.inc10yr_pnwrs/20.,2) AS dg, 
ht AS ht, 
tree.inc5yrht_pnwrs AS htg, 
CASE WHEN actualht < ht THEN actualht ELSE NULL END AS httopk, 
cr AS crratio, 
CASE WHEN actualht < ht THEN 96 ELSE NULL END AS damage1, 
totage AS age
FROM tree, subplot, plot
WHERE plot.cn = subplot.plt_cn AND tree.plt_cn = subplot.plt_cn AND 
subplot.subp = tree.subp
'''

treeinit = pd.read_sql(sql=treeinit_sql, con=pg_engine)

# convert some data types to integers
convert_cols = ['stand_id', 'plot_id', 'species']
treeinit[convert_cols] = treeinit[convert_cols].astype('int64')

treeinit.head()

,stand_id,plot_id,tree_id,tree_count,history,species,dbh,dg,ht,htg,httopk,crratio,damage1,age
0,685690010497,4,536,6.018046,1.0,202,5.0,NaN,35.0,NaN,NaN,40.0,NaN,NaN
1,685690010497,4,538,NaN,NaN,361,NaN,NaN,30.0,NaN,15.0,NaN,96.0,NaN
2,685688010497,2,552,6.018046,1.0,202,9.6,NaN,69.0,NaN,NaN,35.0,NaN,NaN
3,685688010497,2,561,NaN,NaN,431,NaN,NaN,30.0,NaN,NaN,NaN,NaN,NaN
4,685688010497,2,560,0.999188,1.0,202,42.3,NaN,123.0,NaN,83.0,65.0,96.0,NaN


In [5]:
treeinit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1148172 entries, 0 to 1148171
Data columns (total 14 columns):
stand_id      1148172 non-null int64
plot_id       1148172 non-null int64
tree_id       1148172 non-null int64
tree_count    1026711 non-null float64
history       983504 non-null float64
species       1148172 non-null int64
dbh           985985 non-null float64
dg            123638 non-null float64
ht            1068213 non-null float64
htg           15120 non-null float64
httopk        122355 non-null float64
crratio       928006 non-null float64
damage1       122355 non-null float64
age           993 non-null float64
dtypes: float64(10), int64(4)
memory usage: 122.6 MB


In [6]:
treeinit.to_sql('fvs_treeinit', pg_engine, index=False, if_exists='replace')